# **kMeans 알고리즘을 적용하여 확인적 군집분석을 수행하시오.**
>변수 설명 ) tot_price : 총구매액, buy_count : 구매횟수,
                   visit_count : 매장방문횟수, avg_price : 평균구매액

In [ ]:
#필요한 모듈 불러오기
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_samples, silhouette_score

## 1) 데이터 불러오기 및 탐색

In [ ]:
sales = pd.read_csv(".../product_sales.csv")
print(sales.info())

In [ ]:
print(sales.shape)
print(type(sales))

## 2) KMeans 를 이용하여 3개의 군집으로 군집화

2-1) KMeans 모델을 생성하세요. 군집은 3개로 군집화하세요.

In [ ]:
#모델 생성


2-2) 모델을 훈련하고 예측치 생성하고 이 결과를 pred라는 변수에 저장하세요.

2-3) 원형 데이터(scale)에 예측치(변수명:cluster)를 추가하세요.

## 3) 군집화 시각화
: 전체 데이터를 2차원으로 축소한 뒤 시각화하세요.

3-1) cluster 변수를 제외한 sales 데이터를 salesDF라는 이름으로 변수에 저장하세요.

3-2) salesDF 데이터를 이용하여 2차원으로 차원축소 한 후 그 결과를 sales에 pca_x, pca_y 이름으로 저장하세요.

In [ ]:
#차원 축소 한 후 변형된 값 임의의 변수에 저장


#sales에 'pca_x', 'pca_y' 이름으로 열 만들어 저장


3-3) pca_x를 x축, pca_y를 y축으로 하여 산점도를 그리세요. 색으로 각 점이 군집에 따라 구분되도록 하세요. (단, 각 군집별로 마커 모양 구분하지 않고 오직 색으로만 구분합니다.)

## 4) 군집평가: 실루엣 계수 구하기

In [ ]:
#시각화를 위하여 사용했던 차원 축소 관련 계수 삭제
sales=sales.drop(['pca_x','pca_y'], axis=1)

#cluster 행을 제외한 순수 데이터 sales_data로 저장
sales_data=sales.drop('cluster', axis=1)

4-1) sales의 모든 데이터에 실루엣 계수를 구하고 실루엣 계수 칼럼을 추가하세요.

In [ ]:
#sales의 모든 데이터에 실루엣 계수 구하기


In [ ]:
#sales에 실루엣 계수 칼럼 추가


4-2) 모든 데이터의 평균 실루엣 계수를 구하세요.

In [ ]:
#모든 데이터의 평균 실루엣 계수 값을 구하기


4-3) 군집별 평균 실루엣 계수를 구하세요.

In [ ]:
#군집별 평균 실루엣 계수 구하기
